The Data sources are:

### 1. Liam's SPSS coded data
**File:** The Loop 2017 Final Interventions.xlsx

Exported as Excel from SPSS, keeping the variable names.

This file contains 1325 entries.
27 have null festival or sample numbers so can't be used, leaving 1298

### 2. Guy's cleaned up lab data
**File:** Loop 2017 Lab fixed data.csv

Saved from: Dropbox/Testing/2017 results processing/Loop 2017 Lab fixed data.xlsm in the ‘Raw Lab Data’ sheet

This file contains 2544 entries
1900 entries start with F
621 entries begin with A (amnesty) so can't be merged
23 Begin with W? so can't be merged

Festival Names

### 3. Boomtown Intervention Questionnaire
**File:** BTReport 2017 - Form responses 3.csv

Exported from: https://docs.google.com/spreadsheets/d/15pdETY0HK-VbBcV-N0swt6ZrRBbeDnZR5RGDzfq95dg

This file contains 194 entries

All the Data will be merged on the festival and sample number giving 1294 results


In [1]:
# Module imports
import os
import pandas as pd

Need to consolidate 'Festival' and 'Sample Number' Columns across all datasets
* Change all columns to have same name
* Ensure sample numbers are all in the same format: FXXXX for FOH samples and AXXXX for Amnesty samples (which have no intervention data/provenance data associated with them).

In [2]:
bt_interventions = '/opt/random/BTReport 2017 - Form responses 3.csv'

In [3]:
spssdata = '/opt/random/The Loop 2017 Final Interventions.xlsx'
spss_df = pd.read_excel(spssdata)

# Remove SurveyID column
spss_df.drop('SurveyID', axis=1, inplace=True)

# Change festival names
spss_df['Festival'].replace(['BoomTown', 'KC', 'SGP'], ['BT2017', 'KC2017', 'SGP2017'], inplace=True)

# Ensure all Sample numbers are consistent
# 1. Delete any rows where SampleNumber or Festival is NA as we can't do anything with it
spss_df.dropna(subset=['SampleNumber', 'Festival'], inplace=True)
# Make all sample numbers a 4-digit code starting with F
spss_df['SampleNumber'] = spss_df['SampleNumber'].apply(lambda x: 'F{:04d}'.format(int(x)))

# Below shows we are left with 1298 datasets
print(len(spss_df))

1298


In [13]:
labdata = '/opt/random/Loop 2017 Lab fixed data.csv'
lab_df = pd.read_csv(labdata, encoding="ISO-8859-1")

# Remame 'Event Name' and 'Sample Number' columns
lab_df.rename(columns={'Event  Name': 'Festival', 'Sample Number': 'SampleNumber'}, inplace=True)
print(len(lab_df))
# Delete any rows where SampleNumber or Festival is NA as we can't do anything with it
lab_df.dropna(subset=['SampleNumber', 'Festival'], inplace=True)
print(len(lab_df))
# Uppercase all sample numbers
labels = ['SampleNumber']
lab_df.loc[:, labels] = lab_df[labels].apply(lambda x: x.str.upper())

# Some sample numbers begin with W or F 
print(len(lab_df[ ~ (lab_df['SampleNumber'].str.startswith('F') | lab_df['SampleNumber'].str.startswith('A')) ]))



2544
2543
23


In [5]:
dft = pd.merge(spss_df, lab_df, how='inner', on=['Festival','SampleNumber'])
print(len(dft))
writer = pd.ExcelWriter('merged.xlsx')
dft.to_excel(writer, 'MergedData', index=False)
writer.save()

1294


In [ ]:
# Find out which entries can't be merged